In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_community.retrievers import WikipediaRetriever
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

custom_client = WikipediaAPIWrapper(lang="en", top_k_results=5, wiki_client=None)
# Create retriever (default: English, top 3 docs)
retriever = WikipediaRetriever(wiki_client=custom_client)

# Get relevant documents for a query
docs = retriever.invoke("Viral Kholi")
print(docs[0].page_content[:400])


Virat Kohli (born 5 November 1988) is an Indian international cricketer who plays ODI cricket for the India national team and is a former captain in all formats. He is a right-handed batsman and occasional right-arm medium pace bowler. Considered one of the greatest all-format batsmen in the history of cricket, he is called the King, the Chase Master, and the Run Machine for his skills, records an


In [5]:
# ! uv pip install lxml

In [17]:
from langchain_community.retrievers import WikipediaRetriever
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

wiki = WikipediaRetriever(top_k_results=3, lang="en", wiki_client=False)
docs = wiki.invoke("Rohit Sharma")

llm = ChatOllama(model="gemma3:4b")

prompt = ChatPromptTemplate.from_template(
    "Answer ONLY using the context below:\n\n{context}\n\nQuestion:\n{question}"
)

chain = prompt | llm 

response = chain.invoke({'context':docs, 'question':'"Who is Rohit Sharma?'}) 
print(response.content)

Rohit Sharma is an Indian film composer, most famously known for being the music director of the film The Kashmir Files. He began his career composing music for TVCs and films, and has since worked on prominent titles in both the film and OTT spaces, including “The Kashmir Files,” “Aspirants,” and “Maharani 2.”


In [2]:
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_core.documents import Document

In [3]:
# Step 1: Your source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [4]:
# Initilize the embedding model
embedding_model = OllamaEmbeddings(model="nomic-embed-text")

# create chroma vector sotre in memory 
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_name="My-collection"
)

In [5]:
# convert vectorstore into a reteriver
retreriver = vectorstore.as_retriever(search_kwargs={"k":2})

In [7]:
query = "What is chroma used for?"
results = retreriver.invoke(query)

In [8]:
for i, doc in enumerate(results):
    print(f"-----------result{i+1}------------")
    print(doc.page_content)

-----------result1------------
Chroma is a vector database optimized for LLM-based search.
-----------result2------------
Embeddings convert text into high-dimensional vectors.


In [9]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [10]:
from langchain_community.vectorstores import FAISS

# Initialize OpenAI embeddings
embedding_model = OllamaEmbeddings(model="nomic-embed-text")

# Step 2: Create the FAISS vector store from documents
vectorstore = FAISS.from_documents(
    documents=docs,
    embedding=embedding_model
)

In [11]:
# Enable MMR in the retriever
retriever = vectorstore.as_retriever(
    search_type="mmr",                   # <-- This enables MMR
    search_kwargs={"k": 3, "lambda_mult": 0.5}  # k = top results, lambda_mult = relevance-diversity balance
)

In [12]:
query = "What is langchain?"
results = retriever.invoke(query)

In [13]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
LangChain is used to build LLM based applications.

--- Result 2 ---
LangChain supports Chroma, FAISS, Pinecone, and more.

--- Result 3 ---
Embeddings are vector representations of text.


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings 
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever